<div style="text-align: right" align="right"><i>Peter Norvig, 1–25 Dec 2021</i></div>

# Advent of Code 2021

I'm going to do [Advent of Code 2021](https://adventofcode.com/) (AoC), but I won't be competing for points. 

I won't explain each puzzle here; you'll have to click on each day's link (e.g. [Day 1](https://adventofcode.com/2021/day/1)) to understand the puzzle. 

Part of the idea of AoC is that you have to make some design choices to solve part 1 *before* you get to see the description of part 2. So there is a tension of wanting the solution to part 1 to provide general components that might be re-used in part 2, without falling victim to [YAGNI](https://en.wikipedia.org/wiki/You_aren%27t_gonna_need_it). In this notebook I won't refactor the code for part 1 based on what I see in part 2.

# Day 0: Preparations

First, imports that I have used in past AoC years:

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median

import functools
import math
import re

cat     = ''.join
flatten = chain.from_iterable
cache   = functools.lru_cache(None)

Now two functions that I will use each day, `parse` and `answer`.  I will start by writing something like this for part 1 of Day 1:

    in1 = parse(1, int)
    def solve_it(numbers) -> int: return ...
    solve(in1)
    
That is, `parse(1, int)` will parse the data file for day 1 in the format of one `int` per line and return those ints as a tuple. Then a new function I define for the day (here, `solve_it`) will (hopefully) compute the correct answer. I'll then submit the answer to AoC and verify it is correct. If it is, I'll move on to solve part 2. When I get them both done, I'll use the function `answers` to assert that the correct answers are computed. So it will look like this:

    in1 = parse(1, int)
    
    def solve_it(numbers) -> int: return ...
    answer(1.1, solve_it(in1), 123)
    
    def solve_it2(numbers) -> int: return ...
    answer(1.2, solve_it2(in1), 123456)
    
For more complex puzzles, I will include some `assert` statements to show that I am getting the right partial results on the small example in the puzzle description.

Here's `parse` and `answer`:

In [2]:
def parse(day, parser=str, sep='\n') -> tuple:
    """Split the day's input file into sections separated by `sep`, and apply `parser` to each."""
    sections = open(f'AOC2021/input{day}.txt').read().rstrip().split(sep)
    return mapt(parser, sections)

def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was expected."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

Two useful parsers are `ints` and `atoms`:

In [3]:
def ints(text: str) -> Tuple[int]:
    """Return a tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall('-?[0-9]+', text))

Atom = Union[float, int, str]

def atoms(text: str, sep=None) -> Tuple[Atom]:
    """Parse text into atoms (numbers or strs)."""
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args):
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

A few additional  utility functions:

In [4]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def multimap(items: Iterable[Tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    "The product of an iterable of numbers." 
    result = 1
    for n in numbers:
        result *= n
    return result

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)
    
def dotproduct(A, B) -> float: return sum(a * b for a, b in zip(A, B))

# [Day 1](https://adventofcode.com/2021/day/1): Sonar Sweep

The input is a list of integers, such as "`148`", one per line.

1. How many numbers increase from the previous number?
2. How many sliding windows of 3 numbers increase from the previous window?

In [5]:
in1 = parse(1, int)

In [6]:
def increases(nums: List[int]) -> int:
    """How many numbers are bigger than the previous one?"""
    return quantify(nums[i] > nums[i - 1] 
                    for i in range(1, len(nums)))

answer(1.1, increases(in1), 1400)

True

In [7]:
def window_increases(nums: List[int], w=3) -> int:
    """How many sliding windows of w numbers have a sum bigger than the previous window?"""
    return quantify(sum(nums[i:i+w]) > sum(nums[i-1:i-1+w])
                    for i in range(1, len(nums) + 1 - w))

answer(1.2, window_increases(in1), 1429)

True

# [Day 2](https://adventofcode.com/2021/day/2): Dive! 

The input is a list of instructions, such as "`forward 10`".

1. Follow instructions and report the product of your final horizontal position and depth.
1. Follow *revised* instructions and report the product of your final horizontal position and depth. (The "down" and "up" instructions no longer change depth, rather they change *aim*, and going forward *n* units both increments position by *n* and depth by *aim* × *n*.)

In [8]:
in2 = parse(2, atoms)

In [9]:
def drive(instructions) -> int:
    """What is the product of position and depth after following instructions?"""
    pos = depth = 0
    for (op, n) in instructions:
        if op == 'forward': pos += n
        if op == 'down':    depth += n
        if op == 'up':      depth -= n
    return pos * depth

answer(2.1, drive(in2), 1670340)

True

In [10]:
def drive2(instructions) -> int:
    """What is rthe product of position and depth after following instructions?
    This time we have to keep track of `aim` as well."""
    pos = depth = aim = 0
    for (op, n) in instructions:
        if op == 'forward': pos += n; depth += aim * n
        if op == 'down':    aim += n
        if op == 'up':      aim -= n
    return pos * depth

answer(2.2, drive2(in2), 1954293920)

True

# [Day 3](https://adventofcode.com/2021/day/3): Binary Diagnostic

The input is a list of bit strings, such as "`101000111100`".

1. What is the power consumption (product of gamma and epsilon rates)?
2. What is the life support rating (product of oxygen and CO2)?

In [11]:
in3 = parse(3, str) # Parse into bit strings, (e.g. '1110'), not binary ints (e.g. 14)

In [12]:
def common(strs, i) -> str: 
    """The bit that is most common in position i."""
    bits = [s[i] for s in strs]
    return '1' if bits.count('1') >= bits.count('0') else '0'

def uncommon(strs, i) -> str: 
    """The bit that is least common in position i."""
    return '1' if common(strs, i) == '0' else '0'

def epsilon(strs) -> str:
    """The bit string formed from most common bit at each position."""
    return cat(common(strs, i) for i in range(len(strs[0])))

def gamma(strs) -> str:
    """The bit string formed from most uncommon bit at each position."""
    return cat(uncommon(strs, i) for i in range(len(strs[0])))

def power(strs) -> int: 
    """Product of epsilon and gamma rates."""
    return int(epsilon(strs), 2) * int(gamma(strs), 2)
    
answer(3.1, power(in3), 2261546)

True

In [13]:
def select(strs, common_fn, i=0) -> str:
    """Select a str from strs according to common_fn:
    Going left-to-right, repeatedly select just the strs that have the right i-th bit."""
    if len(strs) == 1:
        return strs[0]
    else:
        bit = common_fn(strs, i)
        selected = [s for s in strs if s[i] == bit]
        return select(selected, common_fn, i + 1)

def life_support(strs) -> int: 
    """The product of oxygen (most common select) and CO2 (least common select) rates."""
    return int(select(strs, common), 2) * int(select(strs, uncommon), 2)
    
answer(3.2, life_support(in3), 6775520)

True

# [Day 4](https://adventofcode.com/2021/day/4): Giant Squid

The first line of the input is a permutation of the integers 0-99. That is followed by bingo boards (5 lines of 5 ints each), each separated by *two* newlines.

1. What will your final score be if you choose the first bingo board to win?
2. What will your final score be if you choose the last bingo board to win?

I'll represent a board as a tuple of 25 ints; that makes `parse` easy: the permutation of integers and the bingo boards can both be parsed by `ints`. 

I'm worried about an ambiguity: what if two boards win at the same time? I'll have to assume Eric arranged it so that can't happen. I'll define `bingo_winners` to return a list of boards that win when a number has just been called, and I'll arbitrarily choose the first of them.

In [14]:
order, *boards = in4 = parse(4, ints, sep='\n\n')

In [15]:
Board = Tuple[int]
Line = List[int]
B = 5
def sq(x, y) -> int: "The index number of the square at (x, y)"; return x + B * y

def lines(square) -> Tuple[Line, Line]:
    """The two lines through square number `square`."""
    return ([sq(x, square // B) for x in range(B)], 
            [sq(square % B, y)  for y in range(B)])

def bingo_winners(boards, drawn, just_called) -> List[Board]:
    """Boards that win due to the number just called."""
    def filled(board, line) -> bool: return all(board[n] in drawn for n in line)
    return [board for board in boards
            if just_called in board
            and any(filled(board, line) for line in lines(board.index(just_called)))]

def bingo_score(board, drawn, just_called) -> int:
    """Sum of unmarked numbers multiplied by the number just called."""
    unmarked = sum(n for n in board if n not in drawn)
    return unmarked * just_called

def bingo(boards, order) -> int: 
    """What is the final score of the first winning board?"""
    drawn = set()
    for num in order:
        drawn.add(num)
        winners = bingo_winners(boards, drawn, num)
        if winners:
            return bingo_score(winners[0], drawn, num)

answer(4.1, bingo(boards, order), 39902)

True

In [16]:
def bingo_last(boards, order) -> int: 
    """What is the final score of the last winning board?"""
    boards = set(boards)
    drawn = set()
    for num in order:
        drawn.add(num)
        winners = bingo_winners(boards, drawn, num)
        boards -= set(winners)
        if not boards:
            return bingo_score(winners[-1], drawn, num)
                
answer(4.2, bingo_last(boards, order), 26936)

True

# [Day 5](https://adventofcode.com/2021/day/5): Hydrothermal Venture

The input is a list of "lines" denoted by start and end x,y points, e.g. "`0,9 -> 5,9`". I'll represent a line as a 4-tuple of integers, e.g. `(0, 9, 5, 9)`.

1. Consider only horizontal and vertical lines. At how many points do at least two lines overlap?
2. Consider all of the lines (including diagonals, which are all at ±45°). At how many points do at least two lines overlap?

In [17]:
in5 = parse(5, ints)

In [18]:
def points(line) -> bool:
    """All the (integer) points on a line."""
    x1, y1, x2, y2 = line
    if x1 == x2:
        return [(x1, y) for y in cover(y1, y2)]
    elif y1 == y2:
        return [(x, y1) for x in cover(x1, x2)]
    else: # non-orthogonal lines not allowed
        return []
    
def cover(x1, x2) -> range:
    """All the ints from x1 to x2, inclusive, with x1, x2 in either order."""
    return range(min(x1, x2), max(x1, x2) + 1)

def overlaps(lines) -> int:
    """How many points overlap 2 or more lines?"""
    counts = Counter(flatten(map(points, lines)))
    return quantify(counts[p] >= 2 for p in counts)

answer(5.1, overlaps(in5), 7436)

True

For part 2 I'll redefine `points` and `overlaps` in a way that doesn't break part 1:

In [19]:
def points(line, diagonal=False) -> bool:
    """All the (integer) points on a line; optionally allow diagonal lines."""
    x1, y1, x2, y2 = line
    if diagonal or x1 == x2 or y1 == y2:
        dx, dy = sign(x2 - x1), sign(y2 - y1)
        length = max(abs(x2 - x1), abs(y2 - y1))
        return [(x1 + k * dx, y1 + k * dy) for k in range(length + 1)]
    else: # non-orthogonal lines not allowed when diagonal is False
        return []
    
def overlaps(lines, diagonal=False) -> int:
    """How many points overlap 2 or more lines?"""
    counts = Counter(flatten(points(line, diagonal) for line in lines))
    return quantify(counts[p] >= 2 for p in counts)

assert points((1, 1, 1, 3), False) == [(1, 1), (1, 2), (1, 3)]
assert points((1, 1, 3, 3), False) == []
assert points((1, 1, 3, 3), True) == [(1, 1), (2, 2), (3, 3)]
assert points((9, 7, 7, 9), True) == [(9, 7), (8, 8), (7, 9)]

answer(5.2, overlaps(in5, True), 21104)

True

# [Day 6](https://adventofcode.com/2021/day/6): Lanternfish

The input is a single line of ints, each one the age of a lanternfish. Over time, they age and reproduce in a specified way.

1. Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?
2. How many lanternfish would there be after 256 days?

In [20]:
in6 = parse(6, ints)[0]

Although the puzzle description treats each fish individually, I won't take the bait (pun intended). Instead, I'll use a `Counter` of fish, and treat all the fish of each age group together. I have a hunch that part 2 will involve a ton-o'-fish.

In [21]:
Fish = Counter # Represent a school of fish as a Counter of their timer-ages

def simulate(fish, days=1) -> Tuple[Fish, int]:
    """Simulate the aging and birth of fish over `days`;
    return the Counter of fish and the total number of fish."""
    for day in range(days):
        fish = Fish({t - 1: fish[t] for t in fish})
        if -1 in fish: # births
            fish[6] += fish[-1]
            fish[8] = fish[-1]
            del fish[-1]
    return fish, sum(fish.values())
        
assert simulate(Fish((3, 4, 3, 1, 2))) == (Fish((2, 3, 2, 0, 1)), 5)
assert simulate(Fish((2, 3, 2, 0, 1))) == (Fish((1, 2, 1, 6, 0, 8)), 6)

answer(6.1, simulate(Fish(in6), 80)[1], 350917)

True

My hunch was right, so part 2 is simple:

In [22]:
answer(6.2, simulate(Fish(in6), 256)[1], 1592918715629)

True

# [Day 7](https://adventofcode.com/2021/day/7): The Treachery of Whales

The input is a single line of integers given the horizontal positions of each member of a swarm of crabs.

1. Determine the horizontal position that the crabs can align to using the least fuel possible. (Each unit of travel costs one unit of fuel.) How much fuel must they spend to align to that position?
2. Determine the horizontal position that the crabs can align to using the least fuel possible. (Now for each crab the first unit of travel costs 1, the second 2, and so on.) How much fuel must they spend to align to that position?

In [23]:
in7 = parse(7, ints)[0]

In [24]:
def fuel_cost(positions) -> int:
    """How much fuel does it cost to get everyone to the best alignment point?"""
    # I happen to know that the best alignment point is the median
    align = median(positions)
    return sum(abs(p - align) for p in positions)

answer(7.1, fuel_cost(in7), 352707)

True

In [25]:
def fuel_cost2(positions) -> int:
    """How much fuel does it cost to get everyone to the best alignment point, 
    with nonlinear fuel costs?"""
    # I don't know the best alignment point, so I'll try all of them
    return min(sum(burn_rate2(p, align) for p in positions)
               for align in range(min(positions), max(positions)))

def burn_rate2(p, align) -> int:
    """The first step costs 1, the second 2, etc. (i.e. triangular numbers)."""
    steps = abs(p - align)
    return steps * (steps + 1) // 2

answer(7.2, fuel_cost2(in7), 95519693)

True

Now that I got the right answer and have some time to think about it, if the travel cost were exactly quadratic, we would be minimizing the sum of square distances, and Legendre and Gauss knew that the **mean**, not the **median**, is the alignment point that does that. What's the mean?

In [26]:
positions = in7
mean(positions)

490.543

Let's sum the burn rates for this alignment point in three ways: rounded down, as is, and rounded up:

In [27]:
sum(burn_rate2(p, 490) for p in positions)

95519693

In [28]:
sum(burn_rate2(p, 490.543) for p in positions)

95519083.0

In [29]:
sum(burn_rate2(p, 491) for p in positions)

95519725

We see that rounding down gives the right answer, keeping the mean as is gives a total fuel cost that is *better* than the correct answer (but is apparently not a legal alignment point), and rounding up does a bit worse.

# [Day 8](https://adventofcode.com/2021/day/8): ???